In [1]:
import numpy 
import pandas as pd


In [2]:
def matrix_factorization(R, P, Q, K, steps=2000, alpha=0.0002, beta=0.02):
    Q = Q.T

    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    # print(R[i][j])
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = numpy.dot(P,Q)
        
        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)

        # 0.001: local minimum
        if step%500:er.append(e)
        if e < 0.001:

            break

    return P, Q.T


In [3]:
er=[]
R = [
     [5,3,0,1,2,5,3,0,1,0],
     [4,0,0,1,0,5,3,2,0,3],
     [1,1,0,5,0,2,4,0,1,2],
     [1,0,0,4,3,1,3,1,1,1],
     [0,1,5,4,5,0,3,5,1,0],
     [2,1,3,0,0,5,1,3,1,2],
     [5,3,0,1,2,5,3,0,1,0],
     [4,0,0,1,0,5,3,2,0,3],
     [1,1,0,5,0,2,4,0,1,2],
     [1,0,0,4,3,1,3,1,1,1],
     [0,1,5,4,5,0,3,5,1,0],
     [2,1,3,0,0,5,1,3,1,2],
    ]

R = numpy.array(R)
# N: num of User
N = len(R)
# M: num of Movie
M = len(R[0])
# Num of Features
K = 3

 
P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)

 

nP, nQ = matrix_factorization(R, P, Q, K)

nR = numpy.dot(nP, nQ.T)

In [4]:
numpy.round(nR),R

(array([[5., 3., 3., 1., 2., 5., 3., 3., 1., 3.],
        [4., 2., 3., 1., 2., 5., 3., 2., 1., 3.],
        [1., 1., 3., 5., 4., 2., 4., 2., 1., 2.],
        [1., 0., 2., 4., 3., 1., 3., 1., 1., 1.],
        [4., 2., 5., 4., 5., 6., 3., 5., 1., 4.],
        [2., 1., 3., 3., 3., 4., 2., 3., 1., 2.],
        [5., 2., 3., 1., 2., 6., 3., 3., 1., 3.],
        [4., 2., 3., 1., 2., 5., 3., 2., 1., 3.],
        [1., 1., 3., 5., 4., 2., 4., 2., 1., 2.],
        [0., 0., 2., 4., 3., 1., 3., 1., 1., 1.],
        [4., 2., 5., 4., 5., 6., 3., 5., 1., 4.],
        [2., 1., 3., 3., 3., 4., 1., 3., 1., 2.]]),
 array([[5, 3, 0, 1, 2, 5, 3, 0, 1, 0],
        [4, 0, 0, 1, 0, 5, 3, 2, 0, 3],
        [1, 1, 0, 5, 0, 2, 4, 0, 1, 2],
        [1, 0, 0, 4, 3, 1, 3, 1, 1, 1],
        [0, 1, 5, 4, 5, 0, 3, 5, 1, 0],
        [2, 1, 3, 0, 0, 5, 1, 3, 1, 2],
        [5, 3, 0, 1, 2, 5, 3, 0, 1, 0],
        [4, 0, 0, 1, 0, 5, 3, 2, 0, 3],
        [1, 1, 0, 5, 0, 2, 4, 0, 1, 2],
        [1, 0, 0, 4, 3, 1, 3, 1, 1, 1]

In [5]:
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds

In [6]:
u,s,v = svds(R.astype('float'),k=3)

In [7]:
u.shape,s.shape,v.shape 

((12, 3), (3,), (3, 10))

In [8]:
s_d = numpy.diag(s)
s_d

array([[ 8.82765703,  0.        ,  0.        ],
       [ 0.        , 12.73610292,  0.        ],
       [ 0.        ,  0.        , 21.88000922]])

In [9]:
r_hat = numpy.dot(numpy.dot(u,s_d),v)
r_hat.round(1),R

(array([[ 4. ,  1.4,  0.1,  1.6,  0.4,  5.2,  3.1,  0.9,  0.8,  2. ],
        [ 3.9,  1.4,  0.4,  1. ,  0.3,  5.3,  2.7,  1.1,  0.7,  1.9],
        [ 1.9,  0.9, -0.7,  4.4,  1.5,  1.8,  3.9, -0.1,  0.9,  1.3],
        [ 0.9,  0.7,  0.5,  4.2,  2.2,  0.8,  3.3,  0.9,  0.8,  0.8],
        [-0.3,  0.8,  5. ,  4.2,  4.8,  0.2,  2.9,  5. ,  1.2,  0.3],
        [ 2.9,  1.2,  2.6, -0.5,  0.9,  4.6,  1.2,  3. ,  0.6,  1.3],
        [ 4. ,  1.4,  0.1,  1.6,  0.4,  5.2,  3.1,  0.9,  0.8,  2. ],
        [ 3.9,  1.4,  0.4,  1. ,  0.3,  5.3,  2.7,  1.1,  0.7,  1.9],
        [ 1.9,  0.9, -0.7,  4.4,  1.5,  1.8,  3.9, -0.1,  0.9,  1.3],
        [ 0.9,  0.7,  0.5,  4.2,  2.2,  0.8,  3.3,  0.9,  0.8,  0.8],
        [-0.3,  0.8,  5. ,  4.2,  4.8,  0.2,  2.9,  5. ,  1.2,  0.3],
        [ 2.9,  1.2,  2.6, -0.5,  0.9,  4.6,  1.2,  3. ,  0.6,  1.3]]),
 array([[5, 3, 0, 1, 2, 5, 3, 0, 1, 0],
        [4, 0, 0, 1, 0, 5, 3, 2, 0, 3],
        [1, 1, 0, 5, 0, 2, 4, 0, 1, 2],
        [1, 0, 0, 4, 3, 1, 3, 1, 1, 1]

In [10]:
q=[5, 3, 0, 1, 2, 5, 3, 0, 1, 0]
numpy.dot(R,v.T)

array([[-0.3527413 , -3.57321135,  7.06420921],
       [ 0.38035366, -3.69369552,  6.74280072],
       [-3.84965167,  0.20997275,  5.71249047],
       [-2.56000861,  2.06904129,  5.1532322 ],
       [ 1.90122748,  6.85627158,  7.13924453],
       [ 3.70223459, -1.8328809 ,  5.81737282],
       [-0.3527413 , -3.57321135,  7.06420921],
       [ 0.38035366, -3.69369552,  6.74280072],
       [-3.84965167,  0.20997275,  5.71249047],
       [-2.56000861,  2.06904129,  5.1532322 ],
       [ 1.90122748,  6.85627158,  7.13924453],
       [ 3.70223459, -1.8328809 ,  5.81737282]])

In [11]:
#Importing required modules
import numpy as np
from sklearn.decomposition import TruncatedSVD
 
#Creating array 
A = np.array([[3,4,3],[1,2,3],[4,2,1]])
 
#Fitting the SVD class
trun_svd =  TruncatedSVD(n_components = 2)
A_transformed = trun_svd.fit_transform(A)
 
#Printing the transformed matrix
print("Transformed Matrix:")
print(A_transformed)

Transformed Matrix:
[[ 5.7942733  -0.46723885]
 [ 3.3604414  -1.58000033]
 [ 4.14622666  1.93351902]]


In [12]:
col = ['movieId','title']
movies = pd.read_csv('u.item',usecols=[0,1],delimiter='|',names=col)
rating = pd.read_csv("u.data", delimiter="\t",usecols=[0,1,2] ,
                     names=["userId", "movieId", "rating"])
newdata =  pd.merge(rating,movies,on='movieId')
newdata.sample(5)

,userId,movieId,rating,title
16230,699,328,2,Conspiracy Theory (1997)
74275,450,372,4,Jeffrey (1995)
59691,796,692,5,"American President, The (1995)"
40934,938,678,3,Volcano (1997)
1795,500,1014,2,Romy and Michele's High School Reunion (1997)


In [13]:
newdata = newdata.dropna(axis=0,subset=['title'])
newdata.isnull().sum()

userId     0
movieId    0
rating     0
title      0
dtype: int64

In [14]:
movies_rating_count = (newdata.groupby('title')['rating'].count().reset_index().rename(columns={'rating':'totalRatingCount'}))

In [15]:
newdata_rating = newdata.merge(movies_rating_count,left_on='title',right_on='title',how='left')


In [16]:
threshold = 100
popular_movies = newdata_rating[newdata_rating['totalRatingCount']>=threshold]

In [17]:
rating_matrix = popular_movies.pivot_table(index='movieId',columns='userId',values='rating').fillna(0.0)
rating_matrix =rating_matrix.apply(lambda x:x-x.mean())

In [18]:
K=5
N,M=rating_matrix.shape
P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)

 

nP, nQ = matrix_factorization(R, P, Q, K)

nR = numpy.dot(nP, nQ.T)

In [19]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

In [20]:
sc= StandardScaler()
sc.fit(rating_matrix)

StandardScaler()

In [21]:
rating_matrix_s = sc.transform(rating_matrix)

In [22]:
svd = TruncatedSVD(n_components=10, n_iter=7, random_state=42)

In [23]:
U = svd.fit_transform(rating_matrix_s)

In [24]:
V=svd.components_
S=svd.singular_values_ 

In [25]:
result = numpy.dot(U,V)

In [ ]:
result

In [27]:

# TRUNCATED SVD

from sklearn.decomposition import TruncatedSVD
import numpy as np

K=300
svd = TruncatedSVD(n_components=K, n_iter=7, random_state=42)
X = rating_matrix
US=svd.fit_transform(X)

V=svd.components_
S=svd.singular_values_ 


# print('u,s,v', US,S,V)
print('X_restored dot way',np.round(np.dot(US,V),1))#,'svdinverse way',np.round(svd.inverse_transform(U),1))

# LINALG SVD


U1,S1,V1=np.linalg.svd(X)


# print('u1,s1,v1 remark negative mirrored',U1[:,:K]*S1[:K],V1[:K,:])
print('X restored u1,s1,v1, 2 components',np.round( np.dot( U1[:,:K]*S1[:K],V1[:K,:] ),1 ) ) 

# sparse svd
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds, eigs


A = csc_matrix(X, dtype=float)

u2, s2, vt2 = svds(A, k=K)


# print('sparse reverses !',u2*s2,vt2)
print('x restored',np.round( np.dot(u2*s2,vt2),1) )
rating_matrix.values



X_restored dot way [[ 3.   3.5 -0.3 ...  4.8 -0.8 -1.3]
 [ 1.2 -0.4 -0.6 ... -0.4 -0.1  4.1]
 [ 1.1 -0.7 -0.1 ... -0.2 -1.  -1.1]
 ...
 [-2.  -0.2 -0.4 ... -0.2 -0.5 -1.3]
 [-2.1 -0.7 -0.3 ... -0.3  3.1  1.3]
 [-1.9 -0.4 -0.4 ... -0.3 -0.4  1. ]]
X restored u1,s1,v1, 2 components [[ 3.   3.5 -0.3 ...  4.8 -0.8 -1.3]
 [ 1.2 -0.4 -0.5 ... -0.3 -0.1  4.1]
 [ 1.1 -0.7 -0.1 ... -0.2 -1.  -1.1]
 ...
 [-2.  -0.3 -0.4 ... -0.2 -0.6 -1.3]
 [-2.1 -0.7 -0.4 ... -0.3  3.1  1.3]
 [-1.9 -0.4 -0.5 ... -0.3 -0.6  1. ]]
x restored [[ 3.   3.5 -0.3 ...  4.8 -0.8 -1.3]
 [ 1.2 -0.4 -0.5 ... -0.3 -0.1  4.1]
 [ 1.1 -0.7 -0.1 ... -0.2 -1.  -1.1]
 ...
 [-2.  -0.3 -0.4 ... -0.2 -0.6 -1.3]
 [-2.1 -0.7 -0.4 ... -0.3  3.1  1.3]
 [-1.9 -0.4 -0.5 ... -0.3 -0.6  1. ]]


array([[ 3.0497076 ,  3.49707602, -0.26900585, ...,  4.77777778,
        -0.75438596, -1.23099415],
       [ 1.0497076 , -0.50292398, -0.26900585, ..., -0.22222222,
        -0.75438596,  3.76900585],
       [ 1.0497076 , -0.50292398, -0.26900585, ..., -0.22222222,
        -0.75438596, -1.23099415],
       ...,
       [-1.9502924 , -0.50292398, -0.26900585, ..., -0.22222222,
        -0.75438596, -1.23099415],
       [-1.9502924 , -0.50292398, -0.26900585, ..., -0.22222222,
         3.24561404,  0.76900585],
       [-1.9502924 , -0.50292398, -0.26900585, ..., -0.22222222,
        -0.75438596,  0.76900585]])